# Eager Execution

TensorFlow's eager execution evaluates operations immediately, without an extra graph-building step. Operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow, debug models, reduce boilerplate code, and is ~~fun~~ amazing!

# Setup and basic usage

In [1]:
from __future__ import division, absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

In [2]:
tf.executing_eagerly()

x = [[2.]]
m = tf.matmul(x, x) 
print("hello, {}".format(m))
# tf.Tensor objects reference concrete values instead of 
# symbolic handles to nodes in a computational graph.           COOL
# Since there isn't a computational graph to build and run later 
# in a session, it's easy to inspect results using print() or a debugger.

# TensorFlow math operations convert Python objects 
# and NumPy arrays to tf.Tensor objects. The tf.Tensor.numpy    THANK GOD
# method returns the object's value as a NumPy ndarray.

hello, [[4.]]


In [3]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

# Broadcasting support
b = tf.add(a, 1) #a+1
print(b)

# Operator overloading is supported
print(a * b)     #a.*b    NEAT!!
print("-------")

# Use NumPy values
import numpy as np
c = np.multiply(a, b)
print(c)

# Obtain numpy value from a tensor:
print(a.numpy())

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)
-------
[[ 2  6]
 [12 20]]
[[1 2]
 [3 4]]


# Eager training

## Automatic differentiation

In [5]:
import tensorflow.contrib.eager as tfe

w = tfe.Variable([[1.0]])
with tfe.GradientTape() as tape:
  loss = w * w

grad = tape.gradient(loss, [w])
print(grad) # getting the hang of it, you can print out details of any
            # tf object
    
# During eager execution, use tfe.GradientTape to trace 
# operations for computing gradients later.

[<tf.Tensor: id=48, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>]


In [10]:
# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES]) #generates 1000 #s 0-1
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

def prediction(input, weight, bias):
  return input * weight + bias

# A loss function using mean-squared error
def loss(weights, biases):
  error = prediction(training_inputs, weights, biases) - training_outputs
  return tf.reduce_mean(tf.square(error)) 
    #handles matrices, very cool

# Return the derivative of loss with respect to weight and bias
def grad(weights, biases):
  with tfe.GradientTape() as tape:
    loss_value = loss(weights, biases) 
  return tape.gradient(loss_value, [weights, biases]) 
    # tape.gradient(squared_error, [theta, biases])
    # returns derivatives of loss for each theta! neat. 
    # Note to self: seperate biases from weights

train_steps = 200
learning_rate = 0.01
# Start with arbitrary values for W and B on the same batch of data
W = tfe.Variable(5.) #theta
B = tfe.Variable(10.) #theta(1) in octave
#standard variable, in this case a decimal

print("Initial loss: {:.3f}".format(loss(W, B)))

for i in range(train_steps):
  dW, dB = grad(W, B)
  W.assign_sub(dW * learning_rate) # W -=('gradient' * alpha)
  B.assign_sub(dB * learning_rate)
    #basically gradient descent for a simple linear classifier
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(W, B)))

print("Final loss: {:.3f}".format(loss(W, B)))
print("W = {}, B = {}".format(W.numpy(), B.numpy()))

# basically in this example: 
# tfe.GradientTape() as tape, 
# can do, tape.gradient(loss_value, [weights, biases]
# which takes: cost_fcn, [theta(2:end), theta(1)]
# returns: vector of derivative (-/+ int)

# *Can also be used to do:
# backpropagation,
# derivatives in respect to things, 
# partial derivatives,
# *And can be:
# overloaded,

# Beautiful. (& complicated!)

Initial loss: 69.959
Loss at step 000: 67.229
Loss at step 020: 30.506
Loss at step 040: 14.117
Loss at step 060: 6.802
Loss at step 080: 3.538
Loss at step 100: 2.081
Loss at step 120: 1.431
Loss at step 140: 1.141
Loss at step 160: 1.011
Loss at step 180: 0.953
Final loss: 0.929
W = 3.0279805660247803, B = 2.0723013877868652


## Build and train models

Eager execution encourages the use of the Keras-style layer classes in the "tf.keras.layers" module. Additionally, the "tf.train.Optimizer" classes provide sophisticated techniques to calculate parameter updates.

In [11]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_shape=(784,)),  # must declare input shape
  tf.keras.layers.Dense(10)
]) #tf includes Keras in it